In [11]:
import tkinter as tk
import PIL as pil
from PIL import Image
from PIL import ImageTk 
from tkinter import filedialog
from tkinter import simpledialog

def nbrCol(matrice):
    return(len(matrice[0]))

def nbrLig(matrice):
    return len(matrice)

def saving(matPix, filename):
    toSave=pil.Image.new("RGBA",(nbrCol(matPix),nbrLig(matPix)))
    for i in range(nbrCol(matPix)):
        for j in range(nbrLig(matPix)):
            toSave.putpixel((i,j),matPix[j][i])
    toSave.save(filename)


def loading(filename):
    toLoad=pil.Image.open(filename)
    mat=[[(255,255,255,255)]*toLoad.size[0] for k in range(toLoad.size[1])]
    for i in range(toLoad.size[1]):
        for j in range(toLoad.size[0]):
            mat[i][j]=toLoad.getpixel((j,i))
    return mat

create=True
nomImgCourante=""

def charger(widg):
    global create
    global photo
    global img
    global canvas
    global dessin
    global nomImgCourante
    filename= filedialog.askopenfile(mode='rb', title='Choose a file')
    img = pil.Image.open(filename)
    nomImgCourante=filename.name
    photo = ImageTk.PhotoImage(img)
    if create:    
        canvas = tk.Canvas(widg, width = img.size[0], height = img.size[1])
        dessin = canvas.create_image(0,0,anchor = tk.NW, image=photo)
        canvas.grid(row=0,column=1,rowspan=4,columnspan=2)
        create=False
        
    else:
        canvas.grid_forget()
        canvas = tk.Canvas(widg, width = img.size[0], height = img.size[1])
        dessin=canvas.create_image(0,0,anchor = tk.NW, image=photo)
        canvas.grid(row=0,column=1,rowspan=4,columnspan=2)

def modify(matrice):
    global imgModif
    global nomImgCourante
    saving(matrice,"modif.png")
    imgModif=ImageTk.PhotoImage(file="modif.png")
    canvas.itemconfigure(dessin, image=imgModif)
    nomImgCourante="modif.png"


def fermer_fenetre():
    affi.destroy()

Ecrire une fonction qui renvoie l'équivalent en entier des `k` derniers bits d'un nombre.

In [12]:
def derniers_bits(x,k): #renvoie les k derniers bits de x
    return x & ((1 <<k)-1)

print(derniers_bits(15,3))

7


# Chiffrement

Donner une fonction qui remplace les `k` derniers bits d'un nombre par les `k` derniers bits d'un autre nombre

In [13]:
def encoder_kbits(x,y,k): #remplace les k derniers bits de x par ceux de y
    return x - derniers_bits (x,k) + derniers_bits(y,k)
    pass

print(encoder_kbits(15,8,3))

8


Créer une fonction qui encode un entier `e` sur 8 bits dans les bits de poids faible d'un pixel : 3 derniers bits dans le canal rouge, 3 bits du milieu dans le canal vert et 2 premiers dans le canal bleu

In [19]:
def encoder_entier(e,pixel):  #encode un entier e sur 8 bits dans les bits de poids faible d'un pixel
    r =encoder_kbits(pixel[0],e, 3)
    v =encoder_kbits(pixel[1],e >> 3, 3)
    b =encoder_kbits(pixel[2],e >> 6, 2)
    return (r,v,b)

Quelle fonction permet de transformer un caractère en entier ?

Créer une fonction qui encode un texte (récupéré dans un champ de texte) dans les bits de poids faibles des pixels d'une matrice de pixels, on remarquera qu'il faut également encoder la taille du message crypté si l'on souhaite pouvoir le décrypter.

In [15]:
def coder():
    texte_a_cacher = entree.get()
    mat=loading(nomImgCourante) 
    taille = len(texte_a_cacher)
    mat[0][0] =encoder_entier(taille,mat[0][0])
    i=1
    j=0
    for elm in texte_a_cacher:
        mat[j][i] =encoder_entier(ord(elm),mat[j][i])
        i += 1
        if i == nbrCol(mat) :
            j+=1
            i=0
    #Choisir un ordre sur les pixels pour encoder le message
    modify(mat)

# Déchiffrement

Faire une fonction qui permet de décoder un entier (=un caractère) caché dans un pixel, ATTENTION à l'ordre des bits, on veut retourner un entier

In [26]:
def decoder_pixel(pixel):
    res= (derniers_bits(pixel[2],2) << 6) +( derniers_bits(pixel[1],3) << 3) + derniers_bits(pixel[0],3)
    return res

Quelle fonction permet d'associer un caractère à un entier ?

In [ ]:
La fonction chr()

Créer une fonction qui permet de décoder un message caché dans une image selon l'algorithme que nous avons utilisé plus haut

In [17]:
def decoder():
    mat=loading(nomImgCourante)
    sortie = ""
    taille_msg=decoder_pixel(mat[0][0])
    i=0
    j=1
    while(len(sortie) < taille_msg) :
        sortie += chr(decoder_pixel[i][j])
        if j == nbrCol(mat):
            i+=1
            j=0
    label_decodage.config(text= sortie)
    

In [27]:
affi=tk.Tk()
affi.title("Stéganographie")

entree = tk.Entry(affi, width = 20  ,font = ("helvetica", "20"))
entree.grid(row = 0, column = 0)


bouton_coder=tk.Button(affi, text="Coder texte",fg="black", width=15, command=coder)
bouton_coder.grid(row=1,column=0)

bouton_decoder=tk.Button(affi,text="Décoder texte",fg="black",  width=15,command=decoder)
bouton_decoder.grid(row=2,column=0)

label_decodage=tk.Label(affi, text = "Décode moi !")
label_decodage.grid(row=3,column=0)

bouton = tk.Button(affi, text="Charger", fg="black", width=15, command=lambda :charger(affi))
bouton.grid(row=5,column=0)

btn = tk.Button(affi, text="Quitter", fg="black", width=8, command=fermer_fenetre)
btn.grid(row=5,column=2, sticky='E')

affi.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\dilou\miniconda3\envs\l1-python\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-27-8555facffac5>", line 17, in <lambda>
    bouton = tk.Button(affi, text="Charger", fg="black", width=15, command=lambda :charger(affi))
  File "<ipython-input-11-cf663f2e175a>", line 51, in charger
    canvas.grid_forget()
  File "C:\Users\dilou\miniconda3\envs\l1-python\lib\tkinter\__init__.py", line 2494, in grid_forget
    self.tk.call('grid', 'forget', self._w)
_tkinter.TclError: can't invoke "grid" command: application has been destroyed
